In [1]:
from deepmodel_util import *
from deepmodel_definition import *
import deepmodel_util
import deepmodel_definition

deepmodel_definition.num_labels = 19#
deepmodel_definition.emb_dim    = 300
deepmodel_definition.hidden_dim = 200

deepmodel_util.max_length_title = 30
deepmodel_util.max_length_body  = 100
deepmodel_definition.max_length_title = 30
deepmodel_definition.max_length_body  = 100

dropout_rate=0.5
process_setting(low =False,old = True,stop = False)

suffix = "180days_all_body_shuffled"
prefix = "/home/t-haohu/IndustryClassifier/Data/"


data_train_body = "{}/middle/train_{}.txt".format(prefix,suffix)
data_test_body  = "{}/middle/test_{}.txt".format(prefix,suffix)


data_title_vocab    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_vocab     = "{}/ready/body_{}.wl".format(prefix,suffix)
data_industry_vocab = "{}/ready/industry_{}.wl".format(prefix,suffix)







title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_vocab).readlines()])}
body_dict  =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_body_vocab ).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_vocab).readlines()])}




deepmodel_util.input_xt = C.input_variable(shape=(deepmodel_util.max_length_title),  dtype=np.float32)
deepmodel_util.input_xb = C.input_variable(shape=(deepmodel_util.max_length_body) ,  dtype=np.float32)
deepmodel_util.input_y  = C.input_variable(shape=(1)               ,  dtype=np.int)

deepmodel_definition.input_xt_one_hot = C.one_hot(deepmodel_util.input_xt, num_classes=len(title_dict)   ,  sparse_output=True)
deepmodel_definition.input_xb_one_hot = C.one_hot(deepmodel_util.input_xb, num_classes=len(body_dict)+1    ,  sparse_output=True) 
deepmodel_definition.input_y_one_hot = C.one_hot(deepmodel_util.input_y  , num_classes=len(industry_dict) ,  sparse_output=True)

deepmodel_util.input_xt_one_hot = deepmodel_definition.input_xt_one_hot
deepmodel_util.input_xb_one_hot = deepmodel_definition.input_xb_one_hot
deepmodel_util.input_y_one_hot = deepmodel_definition.input_y_one_hot

deepmodel_util.test_data  = load_data_body(data_test_body,title_dict,body_dict,industry_dict)
deepmodel_util.train_data = load_data_body(data_train_body,title_dict,body_dict,industry_dict)



#embedding_title = load_embedding(data_title_vocab,"word2vec_title.model")
#embedding_body = load_embedding(data_body_vocab,"word2vec_body.model")



    

In [2]:
train_body(create_model_cnn_with_body(),deepmodel_util.train_data,num_epochs=20,learning_rate=[5e-4*30]*2+[1e-4*30],batch_size = 30,tag = "rcnn_body")

[Output('classify', [#], [19]), Output('Splice695_Output_0', [#], [300 x 1 x 1]), Output('pooling_t_1', [#], [50 x 1 x 1]), Output('Block174_Output_0', [#], [50 x 29 x 1]), Output('embed', [#], [30 x 300]), Output('OneHotOp6_Output_0', [#], [30 x 56957]), Output('pooling_t_2', [#], [50 x 1 x 1]), Output('Block111_Output_0', [#], [50 x 30 x 1]), Output('pooling_t_3', [#], [50 x 1 x 1]), Output('Block237_Output_0', [#], [50 x 28 x 1]), Output('pooling_b_1', [#], [50 x 1 x 1]), Output('Block363_Output_0', [#], [50 x 99 x 1]), Output('embed', [#], [100 x 300]), Output('OneHotOp9_Output_0', [#], [100 x 216867]), Output('pooling_b_2', [#], [50 x 1 x 1]), Output('Block300_Output_0', [#], [50 x 100 x 1]), Output('pooling_b_3', [#], [50 x 1 x 1]), Output('Block426_Output_0', [#], [50 x 98 x 1])]
data_size:  794568 batch_size:  30 num_batches_per_epoch:  26486
Learning rate per minibatch: 0.015
1000 34.09126019477844


TypeError: CNTK Function expected 2 arguments, got 1

In [ ]:
train_body(create_model_rcnn_body(),deepmodel_util.train_data,num_epochs=20,learning_rate=[5e-4*30]*2+[1e-4*30],batch_size = 30,tag = "rcnn_body")

In [78]:
#test code 
#run the first cell first to load dict
from cntk import load_model

suffix = "180days_all_body_shuffled"
prefix = "/home/t-haohu/IndustryClassifier/Data/"


data_title_vocab    = "{}/ready/title_{}.wl".format(prefix,suffix)
data_body_vocab     = "{}/ready/body_{}.wl".format(prefix,suffix)
data_industry_vocab = "{}/ready/industry_{}.wl".format(prefix,suffix)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_vocab).readlines()])}
body_dict  =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_body_vocab ).readlines()])}
industry_dict =  { x:i for i,x in enumerate([x.strip("\n") for x in open(data_industry_vocab).readlines()])}

model = load_model("model/180days_all_body_shuffled/rcnn_body_acc0.871.dnn")
data_test_body = "1day_sample_gt_body.txt"
test_data  = load_data_body(data_test_body,title_dict,body_dict,industry_dict,raw=True)
test_body(30,model,test_data)
print("OK")

data_size:  361 batch_size:  30 num_batches_per_epoch:  13
Precision:0.5033521453952122 Recall:0.6820123936735778 Acc:0.4515235457063712
OK


/home/t-haohu/anaconda3/envs/cntk-py35/lib/python3.5/site-packages/ipykernel_launcher.py:223: RuntimeWarning: invalid value encountered in true_divide


In [5]:
#predict code
from cntk import load_model
from data_processor import *
process_setting(low=False,old = True,stop = False)
deepmodel_util.batch_size = 20
#deepmodel_util.input_xt = C.input_variable(**Sequence[Tensor[1]])
    
model_file ="model/180days_all_body_shuffled/cnn_body_acc0.867.dnn"
suffix_file = "180days_all_shuffled"
industry_file = "180days_all_shuffled"

model = load_model(model_file)

data_industry_vocab = "{}/ready/industry_{}.wl".format(prefix,industry_file)
data_title_vocab    = "{}/ready/title_{}.wl".format(prefix,suffix_file)
data_body_vocab     = "{}/ready/body_{}.wl".format(prefix,suffix_file)

title_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_title_vocab).readlines()])}
body_dict =     { x:i for i,x in enumerate([x.strip("\n") for x in open(data_body_vocab).readlines()])}

inference_body(model,"Data/middle/1day_measure_sample_valid.txt","val/dyn_cnn_1day_measure_{}.txt".format(suffix),title_dict,body_dict,data_industry_vocab)


FileNotFoundError: [Errno 2] No such file or directory: '/home/t-haohu/IndustryClassifier/Data//ready/body_180days_all_shuffled.wl'